# Cоздание моделей -- дефекты стали

In [2]:
import numpy as np
import pandas as pd

In [3]:
num_data = pd.read_csv("data.csv")

In [4]:
num_data.describe()

,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,...,V25,V26,V27,V28,V29,V30,V31,V32,V33,Class_b'2'
count,1.941000e+03,1.941000e+03,1.941000e+03,1.941000e+03,1.941000e+03,1.941000e+03,1.941000e+03,1941.000000,1941.000000,1.941000e+03,...,1.941000e+03,1.941000e+03,1.941000e+03,1.941000e+03,1.941000e+03,1.941000e+03,1.941000e+03,1.941000e+03,1.941000e+03,1941.000000
mean,2.928564e-17,-8.785691e-17,1.464282e-17,-7.321409e-18,2.928564e-17,7.321409e-17,4.392845e-17,0.000000,0.000000,-4.237265e-16,...,-6.955338e-17,1.519192e-16,1.171425e-16,8.785691e-17,1.171425e-16,2.928564e-17,-1.464282e-17,1.464282e-16,-8.785691e-17,0.346728
std,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000,1.000000,1.000000e+00,...,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,0.476051
min,-1.096882e+00,-1.233783e+00,-9.264019e-01,-9.264194e-01,-3.660430e-01,-3.647141e-01,-1.921906e-01,-0.402234,-2.631106,-4.986023e+00,...,-2.144852e+00,-5.831911e+00,-1.374040e+00,-2.976055e-01,-3.293228e-01,-5.021236e-01,-1.962229e-01,-1.707256e-01,-5.109540e-01,0.000000
25%,-9.989348e-01,-8.559907e-01,-6.646265e-01,-6.646367e-01,-3.501775e-01,-3.215547e-01,-1.640535e-01,-0.384135,-0.670583,-3.313743e-01,...,-8.317313e-01,-4.281529e-01,-9.934266e-01,-2.976055e-01,-3.293228e-01,-5.021236e-01,-1.962229e-01,-1.707256e-01,-5.109540e-01,0.000000
50%,-2.614528e-01,-3.033684e-01,-2.516411e-01,-2.516653e-01,-3.327642e-01,-2.850352e-01,-1.359164e-01,-0.365240,0.169642,-1.708692e-01,...,2.378344e-02,-1.139334e-02,-2.330830e-01,-2.976055e-01,-3.293228e-01,-5.021236e-01,-1.962229e-01,-1.707256e-01,-5.109540e-01,0.000000
75%,9.254323e-01,9.124006e-01,3.000082e-01,2.999821e-01,-2.073884e-01,-9.247802e-02,7.972909e-05,-0.240685,0.667552,5.246530e-01,...,8.551401e-01,4.349427e-01,1.220732e+00,-2.976055e-01,-3.293228e-01,-5.021236e-01,-1.962229e-01,-1.707256e-01,-5.109540e-01,1.000000
max,2.177615e+00,2.200513e+00,6.388546e+00,6.388491e+00,2.916945e+01,3.431882e+01,4.236755e+01,22.223784,3.686136,6.570346e+00,...,1.813676e+00,5.198773e+00,1.221321e+00,3.358422e+00,3.034970e+00,1.990516e+00,5.093620e+00,5.854335e+00,1.956115e+00,1.000000


In [5]:
target = "Class_b\'2\'"
pos = num_data[target].mean()
neg = 1 - pos

In [6]:
m = num_data.size / 2
weight_for_0 = (1 / neg) 
weight_for_1 = (1 / pos) 

class_weight = {0 : weight_for_0, 1 : weight_for_1}

print(weight_for_0, weight_for_1)

1.5307570977917981 2.884101040118871


## Поделить на train, test

In [7]:
from sklearn.model_selection import train_test_split
import tensorflow as tf

from keras import models
from keras import layers
from keras import optimizers 
from keras import metrics

In [8]:
def stratify_sample(data, cut, target):
    assert(cut) >= 0
    return data.groupby(target, group_keys=False).apply(lambda x: x.sample(frac=1/cut))

In [9]:
## this function is task-specific
def model_train(model, split, epochs, random_target=False):
    model.compile(optimizer=optimizers.Adam(), 
              loss='binary_crossentropy', 
              ) 
    
    x_train, x_test, y_train, y_test = split

    if random_target:
        y_train = np.random.randint(2, size = len(y_train))
        y_test = np.random.randint(2, size = len(y_test))  

    # batch_size made bigger to ensure some important cases are included    
    history = model.fit(x_train, y_train,
                    epochs=epochs,
                    batch_size=min(32, len(x_train)),  
                    validation_data=(x_test, y_test), 
                    class_weight=class_weight)
    return history, model

In [10]:
def sample(data, cut, target):
    assert(cut > 0)
    return data.groupby(target, group_keys=False).apply(lambda x: x.sample(frac=1/cut))    

In [11]:
def get_train_test_split(data, target, cut, random_state = None):
    num_data_new = sample(data, cut, target)
    
    train, test = train_test_split(num_data_new, test_size=0.2, random_state=random_state) 

    y_train = np.array(train[target])
    y_test = np.array(test[target])

    x_train = np.array(train.drop(columns=[target]))
    x_test = np.array(test.drop(columns=[target]))
    
    return x_train, x_test, y_train, y_test 

## Модель сети

In [12]:
params = [15, 30, 30, 1]

In [13]:
input_size = len(num_data.columns) - 1

In [71]:
input_size

33

In [72]:
def make_model(input_size, parameters):
    model = models.Sequential()

    model.add(layers.Dense(parameters[0], activation='relu', input_shape=(input_size,))) # , kernel_regularizer="l2"
    for i in range(1, len(parameters)-1):
        model.add(layers.Dense(parameters[i], activation="relu")) #, kernel_regularizer="l1")
    model.add(layers.Dense(parameters[-1], activation='sigmoid')) # , kernel_regularizer="l1"
    
    model.compile(optimizer=optimizers.Adam(), 
              loss='binary_crossentropy', 
              metrics=["accuracy"]
              )
    
    return model

In [73]:
# METRICS = [metrics.Precision(name="precision"), metrics.BinaryAccuracy(name="accuracy")]

In [74]:
volume = len(num_data)

In [75]:
# cuts = np.linspace(start=1, stop=volume/2, num = 24)

In [86]:
# cuts = list(range(1, 25))
cuts = [1, 2, 3, 4, 5, 6, 7, 16, 32, 64, 128, 200, 300, 400, 512, 670]

In [87]:
# data_vol = [volume, 970, 647, 485, 388, 323, 277, ]
# cuts = list(map(lambda x: int(volume / x), data_vol))

In [88]:
print("Cuts are {}".format(cuts))
print("Volume of data is {}".format(list(map(lambda x: int(volume / x), cuts))))

Cuts are [1, 2, 3, 4, 5, 6, 7, 16, 32, 64, 128, 200, 300, 400, 512, 670]
Volume of data is [1941, 970, 647, 485, 388, 323, 277, 121, 60, 30, 15, 9, 6, 4, 3, 2]


In [89]:
get_model_directory = lambda series, cut: "models/{}_series/{}_cut/".format(series, cut)
get_model_name = lambda number:  "m_{}.h5".format(number)

## Логи

In [90]:
import datetime 
# для логов
def log_preamble(log_file_name):
    global tries, epochs, series, file_name
    time_stamp = datetime.datetime.now()

    with open(log_file_name, "a") as log_file:
        log_file.write(f"\n\nExecuted on time is {datetime.datetime.now()}\n")
        log_file.write(f"Tries: {tries}, epochs: {epochs}, series = {series}\n")
        # log_file.write(f"Data file is {file_name}\n")
        log_file.write(f"Network configuration is {params}\n")
    return time_stamp

def log_final(log_file_name, time_stamp):
    time_stamp_new = datetime.datetime.now()
    with open(log_file_name, "a") as log_file:
        log_file.write(f"Finished successfully at {time_stamp_new}\n")
        log_file.write(f"Total time = {(time_stamp_new - time_stamp).total_seconds()}\n")

## Обучение

In [91]:
# путь к файлу модели
model_name = lambda series_num, cut_val, model_num: get_model_directory(series_num, cut_val) + get_model_name(model_num)

In [92]:
from keras.callbacks import ModelCheckpoint
# В теле этой функции задаются параметры обучения, чекпоинты
def model_train(model_name, data_split, epochs, random_target=False):
    model = make_model(input_size, params)

    early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss', 
    verbose=1,
    patience=15,
    mode='auto',
    restore_best_weights=True)

    x_train, x_test, y_train, y_test = data_split

    if random_target:
        y_train = np.random.randint(2, size = len(y_train))
        y_test = np.random.randint(2, size = len(y_test))  

    history = model.fit(x_train, y_train,
                    epochs=epochs,
                    batch_size=min(32, len(x_train)),  
                    validation_data=(x_test, y_test), 
                    class_weight=class_weight, 
                    callbacks = [early_stopping]) 
    return history, model

In [93]:
series = [20] # набор серий. в каждой серии модели обучаются на одном и том же датасете
tries = 30 # количество моделей в каждой серии
epochs = 40

histories = [[[] for _ in cuts] for _ in series] # для сохранения истории обучения 

log_file = "model_log.txt"
t = log_preamble(log_file)

for i, s in enumerate(series):
    for cut_number, cut in enumerate(cuts):
        split = get_train_test_split(num_data, target, cut)
        for number in range(tries):
            model_name_current = model_name(s, cut, number)
            history, model = model_train(model_name_current, split, epochs) 
            
            model.save(model_name_current)
            histories[i][cut_number].append(history)

log_final(log_file, t)

Epoch 1/40
49/49 [==============================] - 5s 34ms/step - loss: 1.2405 - accuracy: 0.6624 - val_loss: 0.5488 - val_accuracy: 0.7686
Epoch 2/40
49/49 [==============================] - 1s 12ms/step - loss: 0.9352 - accuracy: 0.8151 - val_loss: 0.3809 - val_accuracy: 0.8766
Epoch 3/40
49/49 [==============================] - 0s 9ms/step - loss: 0.5307 - accuracy: 0.9265 - val_loss: 0.1751 - val_accuracy: 0.9614
Epoch 4/40
49/49 [==============================] - 1s 14ms/step - loss: 0.1905 - accuracy: 0.9794 - val_loss: 0.0698 - val_accuracy: 0.9794
Epoch 5/40
49/49 [==============================] - 1s 16ms/step - loss: 0.0697 - accuracy: 0.9923 - val_loss: 0.0425 - val_accuracy: 0.9871
Epoch 6/40
49/49 [==============================] - 0s 8ms/step - loss: 0.0314 - accuracy: 0.9961 - val_loss: 0.0281 - val_accuracy: 0.9897
Epoch 7/40
49/49 [==============================] - 0s 7ms/step - loss: 0.0146 - accuracy: 1.0000 - val_loss: 0.0236 - val_accuracy: 0.9897
Epoch 8/40
49/49

KeyboardInterrupt: 

## Графы

In [ ]:
def display_loss_graph(history):
    history_dict = history.history
    loss_values = history_dict['loss'] 
    val_loss_values = history_dict['val_loss']
    epochs = range(1, len(loss_values)+1)
    plt.plot(epochs, loss_values, 'bo', label='Training loss')
    plt.plot(epochs, val_loss_values, 'b', label='Validation loss')
    plt.title('Training and validation loss')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.legend()
    plt.show()

def display_accuracy_graph(history):
    history_dict = history.history
    acc = history_dict["accuracy"]
    val_acc = history_dict['val_accuracy']
    epochs = range(1, len(acc)+1)
    plt.plot(epochs, acc, 'bo', label='Training accuracy')
    plt.plot(epochs, val_acc, 'b', label='Validation accuracy')
    plt.title('Training and validation accuracy')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.legend()
    plt.show()

In [ ]:
import pickle
import os

# сохраняет все данные о серии
for s in range(len(series)):
    for i, cut in enumerate(log_cuts):
        for x in range(tries):
            filename = './histories/{}_series/{}_cut/m_{}'.format(s, i, x)

            os.makedirs(os.path.dirname(filename), exist_ok=True)
            with open(filename, "wb") as file:
                pickle.dump(histories[s][i][x], file)

IndexError: list index out of range

In [ ]:
accuracy_score = [[[-1 for _ in range(tries)] for _ in log_cuts] for _ in series]

In [ ]:
for s in range(len(series)):
    for i, cut in enumerate(log_cuts):
        for x in range(tries):
            history_dict = histories[s][i][x].history["val_accuracy"]
            accuracy_score[s][i][x] = np.max(history_dict)

In [ ]:
plt.boxplot(accuracy_score[0])

In [ ]:
# tries = 10
# epochs = 30
# series = [0, 1]
# # histories = [[[] for cut in log_cuts] for i in series] 
# # before compiling
# initial_model = model
# log_file = open("model_log.txt", "w")
# log_file.write("Tries: {}, epochs: {}, series = {}".format(tries, epochs, series))
# for i in series:
#     for cut in log_cuts:
#         model_dir = get_model_directory(i, cut)
#         split = get_train_test_split(num_data, target, cut)
#         for number in range(tries):
#             # wipe the model every time
#             new_model = models.clone_model(initial_model)   
#             # recompile, train and save
#             history, model = model_train(new_model, split, epochs) 
            
#             model_name = model_dir + get_model_name(number)
#             models.save_model(model, filepath = model_name, include_optimizer = False)
#             # histories[i][cut].append(history)
#             # display_history(history)   

Epoch 1/30
49/49 [==============================] - 1s 5ms/step - loss: 1.4323 - val_loss: 0.7054
Epoch 2/30
49/49 [==============================] - 0s 2ms/step - loss: 1.2602 - val_loss: 0.6442
Epoch 3/30
49/49 [==============================] - 0s 3ms/step - loss: 1.1149 - val_loss: 0.5656
Epoch 4/30
49/49 [==============================] - 0s 2ms/step - loss: 0.8997 - val_loss: 0.4399
Epoch 5/30
49/49 [==============================] - 0s 2ms/step - loss: 0.6534 - val_loss: 0.3040
Epoch 6/30
49/49 [==============================] - 0s 2ms/step - loss: 0.4109 - val_loss: 0.1631
Epoch 7/30
49/49 [==============================] - 0s 3ms/step - loss: 0.2144 - val_loss: 0.0731
Epoch 8/30
49/49 [==============================] - 0s 3ms/step - loss: 0.1052 - val_loss: 0.0384
Epoch 9/30
49/49 [==============================] - 0s 2ms/step - loss: 0.0577 - val_loss: 0.0231
Epoch 10/30
49/49 [==============================] - 0s 3ms/step - loss: 0.0367 - val_loss: 0.0158
Epoch 11/30
49/49 [

In [ ]:
# import pickle
# import os

# for i, h in enumerate(histories):
#     filename = './histories/model_{}'.format(i)

#     os.makedirs(os.path.dirname(filename), exist_ok=True)
#     with open(filename, "wb") as file:
#         pickle.dump(h, file)

In [ ]:
# models.save_model(model, filepath ="./models/RANDOM.h5", include_optimizer = False)

In [ ]:
# _random_histories = []
# before compiling
# initial_model = model

# for i in range(5):
    # wipe the model every time
    # new_model = models.clone_model(initial_model)   
    # recompile, train and save
    # history = model_train_save(new_model, cuts[j], i, j, random_target=True) 
    # _random_histories.append(history)